In [1]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Rating scraps.csv')
df

,Unnamed: 0,Reviews,Ratings
0,0,It's mobile is amazing because this phone is b...,5
1,1,Best smartphone in this price range. I special...,5
2,2,Camera dcent display Super Audio best proximit...,3
3,3,I am type Reviews after one month using this p...,5
4,4,"It's a beauty. Superb value for money.Slim, li...",5
...,...,...,...
7346,7346,Wow 😳READ MORE,5
7347,7347,No doubt....The best tp link wifi router with ...,5
7348,7348,AwesomeREAD MORE,5
7349,7349,Good productREAD MORE,5


In [3]:
df.shape

(7351, 3)

In [4]:
df.isnull().sum()

Unnamed: 0    0
Reviews       0
Ratings       0
dtype: int64

In [5]:
df['Ratings'].value_counts()

5    4729
4    1605
1     464
3     385
2     168
Name: Ratings, dtype: int64

In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
for i in range(len(df)):
    df['Reviews'][i]=df['Reviews'][i][:-9]

<ipython-input-7-4f622dcb98f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reviews'][i]=df['Reviews'][i][:-9]


In [8]:
sw=stopwords.words('english')
ps=PorterStemmer()

In [9]:
def clean_text(sample):
    sample=sample.lower()
    sample=re.sub("[^a-zA-z]+",' ',sample)
    sample=sample.split()
    sample=[ps.stem(s) for s in sample if s not in sw]
    sample=" ".join(sample)
    
    return sample

In [10]:
df['Cleaned_Review']=df['Reviews'].apply(clean_text)
df

,Reviews,Ratings,Cleaned_Review
0,It's mobile is amazing because this phone is b...,5,mobil amaz phone best price bettri good perfor...
1,Best smartphone in this price range. I special...,5,best smartphon price rang special purchas phon...
2,Camera dcent display Super Audio best proximit...,3,camera dcent display super audio best proxim s...
3,I am type Reviews after one month using this p...,5,type review one month use phone write best mob...
4,"It's a beauty. Superb value for money.Slim, li...",5,beauti superb valu money slim light great desi...
...,...,...,...
7346,Wow 😳,5,wow
7347,No doubt....The best tp link wifi router with ...,5,doubt best tp link wifi router perfect price
7348,Awesome,5,awesom
7349,Good product,5,good product


In [12]:
df['Reviews'][0]

"It's mobile is amazing because this phone is best on price .bettry is good and performance is very good👍 😊😊😇 and my gf Loved it and I love infinix"

In [13]:
clean_text(df['Reviews'][0])

'mobil amaz phone best price bettri good perform good gf love love infinix'

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

svc=SVC()
dtc=DecisionTreeClassifier()
rfc=RandomForestClassifier()


In [15]:
tfid=TfidfVectorizer()

X=tfid.fit_transform(df['Cleaned_Review'])
Y=df['Ratings']

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [16]:

svc.fit(x_train,y_train)
svc.score(x_train,y_train)

0.9552964042759962

In [17]:
y_pred=svc.predict(x_test)
print("Accuracy score :",accuracy_score(y_test,y_pred))
print("Classification report :",classification_report(y_test,y_pred))

Accuracy score : 0.9206708975521306
Classification report :               precision    recall  f1-score   support

           1       0.93      0.86      0.89       146
           2       1.00      0.72      0.84        53
           3       1.00      0.74      0.85       102
           4       0.96      0.78      0.86       458
           5       0.90      0.99      0.95      1447

    accuracy                           0.92      2206
   macro avg       0.96      0.82      0.88      2206
weighted avg       0.93      0.92      0.92      2206



In [18]:
#Decision tree
dtc.fit(x_train,y_train)
dtc.score(x_train,y_train)

0.9652089407191448

In [19]:
preddtc=dtc.predict(x_test)
print('Accuracy score :',accuracy_score(y_test,preddtc))
print('Classification report :',classification_report(y_test,preddtc))

Accuracy score : 0.9465095194922938
Classification report :               precision    recall  f1-score   support

           1       0.91      0.94      0.93       146
           2       0.92      0.83      0.87        53
           3       0.96      0.97      0.97       102
           4       0.97      0.83      0.90       458
           5       0.94      0.99      0.96      1447

    accuracy                           0.95      2206
   macro avg       0.94      0.91      0.92      2206
weighted avg       0.95      0.95      0.95      2206



In [20]:
#Random forest
rfc.fit(x_train,y_train)
rfc.score(x_train,y_train)

0.9652089407191448

In [21]:
predrfc=rfc.predict(x_test)
print('Accuracy score :',accuracy_score(y_test,predrfc))
print('Classification report :',classification_report(y_test,predrfc))

Accuracy score : 0.9465095194922938
Classification report :               precision    recall  f1-score   support

           1       0.94      0.92      0.93       146
           2       0.92      0.83      0.87        53
           3       0.99      0.97      0.98       102
           4       0.98      0.83      0.90       458
           5       0.94      0.99      0.96      1447

    accuracy                           0.95      2206
   macro avg       0.95      0.91      0.93      2206
weighted avg       0.95      0.95      0.95      2206



Random forest model is the best model with accuracy score of 94.65%